In [12]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [13]:
# !pip install plaidml-keras plaidbench

In [14]:
try :
    import string
    import pandas as pd
    import numpy as np
    import re
    from matplotlib import pyplot as plt
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import LabelEncoder

    from keras.layers import Input, Dense
    from keras.models import Model
    #from featureExtraction import *

    from random import shuffle
    from sklearn.metrics.pairwise import euclidean_distances
    from sklearn.metrics.pairwise import cosine_similarity
    from random import seed
    import csv
    import glob
    from data_extraction import *
    from FeatureValues import *
    from featureExtraction import *
    from name_similarity import *
    from all_cosine_calculation import *
    from only_cosine_calculation import *
    from only_euclidean_calculation import *
    from jaccard_new import *
    import itertools
    import time
    import warnings
    import shutil
    from shutil import make_archive
    from keras.models import load_model, model_from_json
    warnings.filterwarnings("ignore")

except ImportError as ie:
    print(ie , ". Either check if the module is valid or install it")

In [15]:
def drop_emp_columns(df):
    
    print("Before dropping empty columns, df,shape is ",df.shape)
    tot = df.shape[0]
    df = df.replace(r'([\t+\s+$\n]+)', "NA", regex=True)
    for i in df.columns:
        NA_count_perc = df[i].eq("NA").sum()/tot
        Nan_count_perc = df[i].isna().sum()/tot
        if(NA_count_perc > 0.30 or Nan_count_perc > 0.30):
            df.drop([i], axis=1,inplace=True)
            print("Column " + i + " dropped with ", np.maximum(NA_count_perc,Nan_count_perc))
            
    print("After dropping empty columns, df,shape is  " ,df.shape)
    
    return df 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 19 12:06:03 2019

@author: ivishnoi
"""



"""
#DQC PIPELINE
0)Extract test data and prepare test data by appending table names to column names
1)Load both datasets(train dataset-CDB/NDB standard columns, test dataset-incoming source)
2)Calculate and store features for both the datasets ( categorical features, probability distribution features).
3)Use these features as feature vectors to calculate a)all_cosine, b)cosine similarity, c)euclidean similarity
4)Calculate jaccard similarity between the base dataset and the new source (many-many)
5)Calculate name similarity between the column names in the two datasets
6)Use weights as calculated originally, to identify all similar columns in the new data source to each of the base columns.

"""

"""Storing results table wise i.e. each table in the new incoming database is treated separately and results for it
    are stored in a different folder for easier result comparison"""
    
    
#MAIN FILE


working_dir='/Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/'

try: 
    os.chdir(working_dir)
except FileNotFoundError as fnfe:
    print(fnfe)
    print("Check your path!!  Please provide a valid directory path!!")

##READING TRAINING DATA SET
start=time.clock()
try:
    df_train=pd.read_csv('TRAIN_DATA.csv',encoding='latin1')
except UnicodeDecodeError as ude:
      print(ude,"\nSome error with Decoding. Most preferred encoding is utf-8.Look at the `List of Python standard encodings at <https://docs.python.org/3/library/codecs.html#standard-encodings>` \n")
except pd.errors.ParserError as pe:
    print(pe,"\nParser Error .Check if the pandas function and your file type is mismatching \nEg. pd.read_csv('xyz.xlsx')....(Just one of the reasons)")

df_train=df_train[0:1000]

#df_train= df_train.add_prefix(database_name+'.') 
#df_train.to_csv('new_train.csv',index=False)
try:
    df_train = drop_emp_columns(df_train) 
except :
    print("Something wrong in drop_emp_columns() function. ")
df_train.head(3)
print("*Train data shape:",df_train.shape)
print("*Reading train data- Done*")
print((time.clock()-start)/60)  


##BUILDING FEATURES FOR TRAINING DATASET

        #CONTENT AND PROBABILITY BASED FEATURES
start=time.clock()
try:
    df_train_features=featureExtraction(df_train)
    df_train_features.head(3)
except :
    print("Problem with featureExtraction() Module")

df_train_features['columnName'] = df_train_features['columnName'].map(lambda x: x.rstrip('.1').rstrip('.2').rstrip('.3'))
df_train_features.drop_duplicates(keep='first', inplace=True)
df_train_features.to_csv('process_files/TRAIN_FEATURES_CONTENT.csv',index=False)
print("*Features for train data-done*")
print((time.clock()-start)/60)  




##EXTRACTING AND PREPARING TEST DATA

start=time.clock()

"""INPUT SHORTHAND DATABASE NAME TO BE APPENDED TO COLUMN NAMES"""

database_name='NDB'



"""OR INPUT THE FOLDER LOCATION WHERE PRE EXTRACTED TEST DATA FILE ARE STORED"""

path = "NDB/"

filenames_test=[f for f in glob.glob(path + "**/*.csv", recursive=True)]

#df2=pd.read_csv(filenames_test[0],encoding='latin1')
#
#for i in range(1,len(filenames_test)):
#    df1=pd.read_csv(filenames_test[i],encoding='latin1')
#    df2=pd.concat([df2,df1],axis=1)
#
#
#    ## ADD DATABASE PREFIX TO COLUMN NAMES
#df2= df2.add_prefix(database_name+'.')  
#filename=database_name+'_alldata.csv'
#df2.to_csv(filename,index=False)
#
#print("*New data extraction and naming-Done*")
#print((time.clock()-start)/60)  

print("filenames_test",filenames_test)

for i in range(0,len(filenames_test)):
    os.chdir(working_dir)
    print(os.getcwd())
    test_filename=filenames_test[i]
    df_test=pd.read_csv(filenames_test[i],encoding='latin1')
    df_test = drop_emp_columns(df_test)
    count_row = df_test.shape[0]
    if(count_row > 900):
        df_test=df_test.add_prefix(database_name+'.')
    else: # if there are less than 900 entries, then dont even consider that database for finding dsimilar columns
        continue   






    start=time.clock()
    
    
    
    df_test=df_test[0:1000]
    df_test = drop_emp_columns(df_test)
    df_test=df_test[[c for c in df_test if df_test[c].isnull().sum() < 900]] ##keeping column if less than 900 nulls
    print("### TEST DATA: ", filenames_test[i])
    print("*Test data shape:",df_test.shape)
    df_test.to_csv('process_files/TEST_DATA.csv',index=False)
    print("*Reading test data-done*")
    print((time.clock()-start)/60)  
    
    
    
    ##BUILDING FEATURES FOR TEST DATA SET
    
    
    start=time.clock()
    df_test_features=featureExtraction(df_test)
    df_test_features['columnName'] = df_test_features['columnName'].map(lambda x: x.rstrip('.1').rstrip('.2').rstrip('.3'))
    df_test_features.drop_duplicates(keep='first', inplace=True)
    df_test_features.to_csv('process_files/TEST_features.csv',index=False)
    
    
    print("*Features for test data-done*")
    print((time.clock()-start)/60)  
    
    
    
    ##CALCULATING FEATURE BASED SIMILARITY SCORES  
    start=time.clock()
    df_test_features.fillna(0, inplace=True)
    
    
    ## ALL COSINE SCORES
    #start=time.clock()
    df_all_cosine=all_cosine_calculation(df_train_features,df_test_features)
    df_all_cosine.to_csv('process_files/all_cosine_scores.csv',index=False)
    #print(time.clock()-start) 
    
    
    
    ## ONLY COSINE SCORES - CATEGORICAL VARIABLES 
    #start=time.clock()
    df_only_cosine=only_cosine_calculation(df_train_features,df_test_features)
    df_only_cosine.to_csv('process_files/only_cosine_scores.csv',index=False)
    #print(time.clock()-start) 
    
    
    
    ## ONLY EUCLIDEAN SCORES - NUMERICAL VARIABLES
    #start=time.clock()
    df_only_euclidean=only_euclidean_calculation(df_train_features,df_test_features)
    df_only_euclidean.to_csv('process_files/only_euclidean_scores.csv',index=False)
    #print(time.clock()-start)
    
    print("*Feature based scores-done*")
    print((time.clock()-start)/60)  
    
    
    
    
    
    ##COMBINING THESE FEATURE BASED SCORES
    
    
    df_train_transposed=df_train.T
    df_train_transposed.to_csv('process_files/TRAIN_TRANSPOSED.csv')
    #doubt, thiswas already done right?
    
        ## Writing 'ColumnName' to the first cell of transpose
    f = open('process_files/TRAIN_TRANSPOSED.csv', 'r')
    reader = csv.reader(f)
    mylist = list(reader)
    f.close()
    mylist[0][0] = 'ColumnName'
    my_new_list = open('process_files/TRAIN_TRANSPOSED.csv', 'w', newline = '')
    csv_writer = csv.writer(my_new_list)
    csv_writer.writerows(mylist)
    my_new_list.close()
    
    df_train_transposed=pd.read_csv('process_files/TRAIN_TRANSPOSED.csv',encoding='latin1')
    
    
    
    df_test_transposed=df_test.T
    df_test_transposed.to_csv('process_files/TEST_TRANSPOSED.csv')
    
        ## Writing 'ColumnName' to the first cell of transpose
    f = open('process_files/TEST_TRANSPOSED.csv', 'r')
    reader = csv.reader(f)
    mylist = list(reader)
    f.close()
    mylist[0][0] = 'ColumnName'
    my_new_list = open('process_files/TEST_TRANSPOSED.csv', 'w', newline = '')
    csv_writer = csv.writer(my_new_list)
    csv_writer.writerows(mylist)
    my_new_list.close()
    
    df_test_transposed=pd.read_csv('process_files/TEST_TRANSPOSED.csv',encoding='latin1')
    
    
    
    ## CALCULATING JACCARD SCORES
    
    df_feature_scores2=pd.read_csv('blank_file.csv',encoding='latin1') # a blank file to write scores to
    
    start=time.clock() 
    df_jaccard_scores=jaccard_scores(df_train_transposed,df_test_transposed,df_feature_scores2)
    df_jaccard_scores.to_csv('process_files/jaccard_scores.csv',index=False)
    
    print("*Jaccard scores-done*")
    print((time.clock()-start)/60)  
    
    
    ##CALCULATING NAME SIMILARITY
    
    start=time.clock() 
    df_name_scores=name_similarity_scores(df_jaccard_scores)
    df_name_scores.to_csv('process_files/jaccard_name_scores.csv',index=False)
    
    
    print("*Name similarity scores-done*")
    print((time.clock()-start)/60)  
    
    ## COMBINING FEATURE SCORES WITH JACCARD AND NAME
    
    ###Merging the scores 
    start=time.clock()
    df_1 = pd.merge(df_name_scores, df_all_cosine, how='left', on=['train_col', 'test_col'])
    df_1.drop_duplicates(keep='first', inplace=True)
    
    
    df_2 = pd.merge(df_1, df_only_cosine, how='left', on=['train_col', 'test_col'])
    df_2.drop_duplicates(keep='first', inplace=True)
    
    df_3 = pd.merge(df_2, df_only_euclidean, how='left', on=['train_col', 'test_col'])
    df_3.drop_duplicates(keep='first', inplace=True)
    
    
    df_3['train_col'] = df_3['train_col'].map(lambda x: x.rstrip('.1').rstrip('.2').rstrip('.3'))
    df_3['test_col'] = df_3['test_col'].map(lambda x: x.rstrip('.1').rstrip('.2').rstrip('.3')) ##removing any redundant columns
    
    df_3.drop_duplicates(keep='first', inplace=True)
    
    print("*Combining scores 1- done*")
    print((time.clock()-start)/60)  
    
    
    
    ## CALCULATING WEIGHTED SCORES
    
    start=time.clock()
    def f1(all_cosine,only_cosine,euclidean,name_sim):
        s=(0.296*name_sim)+(0.277*all_cosine)+(0.07*only_cosine)+(0.356*euclidean)
        return s
    
    def f2(all_cosine,only_cosine,euclidean):
        s=(0.419*all_cosine)+(0.145*only_cosine)+(0.435*euclidean)
        return s
    
    
    
    df_all_scores=df_3
    
    df_all_scores['weighted_name_score'] = df_all_scores.apply(lambda x: f1(x.all_cosine_score, x.only_cosine_score, x.only_euclidean_score, x.name_cosine_scores_new), axis=1)
    df_all_scores['weighted_noname_score'] = df_all_scores.apply(lambda x: f2(x.all_cosine_score, x.only_cosine_score, x.only_euclidean_score), axis=1)
    
    
    df_all_scores.to_csv('process_files/ALL_SCORES.csv',index=False)
    
    
    print("*Weighted scores calculation-Done*")
    print((time.clock()-start)/60)  
    
    ##DISPLAYING AND STORING RESULTS USING WEIGHTS
    
    # keeping threshold as 0.9 for both weighted scores and 0.2 for jaccard
    df_result=pd.read_csv('blank_file.csv',encoding='latin1')
    
    
    base_cols=list(df_train_transposed['ColumnName'].values)
    
    df_test.columns = [x.lower() for x in df_test.columns]
    
    
    
    # defining the name of the directory to be created
    
    
    
    path =test_filename
    b=path
    print(b)
    c=b[b.find('/')+1:b.find('.')]
    print("Filename : ",c)
    
    path=database_name+'.'+c
    print(path)
    
    
    path = working_dir+ "NDB_Results/" + path
    
    try:  
        os.mkdir(path)
    except OSError:  
        print ("Creation of the directory %s failed" % path)
    else:  
        print ("Successfully created the directory %s " % path)
    
    
    os.chdir(path)
    
    
    
    
    start=time.clock()
    for c in range(0,len(base_cols)):
        base_column1=base_cols[c]
        #(print(base_column1))
        input_column=base_column1.lower()
        date_col_train=df_train_features.date.iloc[c]
        #print("INPUT COLUMN IS:",input_column)
        
        
        output_col_names=[]
        
        for i in range(0,len(df_all_scores)):
            train_column1=df_all_scores.train_col.iloc[i]
            train_column=train_column1.lower()
            #train_column=train_column2.replace('.csv','')
            if(train_column==input_column):
                test_column1=df_all_scores.test_col.iloc[i]
                test_column2=test_column1.lower()
                #test_column3=test_column2.replace('.csv','')
                test_column=test_column2.rstrip()
                #print("COMPARED TO:",test_column)
                
                if(date_col_train==1):
                    for j in range(0,len(df_test_features)):
                        testcol=df_test_features.columnName.iloc[j]
                        testcol=testcol.lower()
                        testcol=testcol.rstrip()
                        if(testcol==test_column):
                            date_col_test=df_test_features.date.iloc[j]
                            if(date_col_test==1):
                                output_col_names.append(test_column)
                    
                    
                    
    
                
                else:
                    name_score=df_all_scores.name_cosine_scores_new.iloc[i]
                    jaccard_score=df_all_scores.jaccard_similarity.iloc[i]
                    weighted_score_name=df_all_scores.weighted_name_score.iloc[i]
                    weighted_score_noname=df_all_scores.weighted_noname_score.iloc[i]
                    
                    
                    
                    
                    
                    if(name_score==1):
                        output_col_names.append(test_column)
                    elif(jaccard_score>0.02):
                        output_col_names.append(test_column)
                    elif(weighted_score_name>=0.9):
                        output_col_names.append(test_column)
                    elif(weighted_score_noname>=0.9):
                        output_col_names.append(test_column)
                
                
        output_col_names=list(set(output_col_names))
        #print(len(output_col_names))
        if(len(output_col_names)==0):
            #print("NEW COLUMN/ NO MATCHES FOUND")
            df_result.loc[1,input_column]='NO MATCHES'
        else:
            #print("\nOUTPUT COLUMNS ARE:")
            for j in range(1,len(output_col_names)):
                df_result.loc[j+1,input_column]=output_col_names[j]
                
            
    
            try:
                df_output=df_test[output_col_names]
                try:
                    df_output.insert(0,input_column, df_train[base_column1])
                except ValueError:
                    input_column1=input_column+'1'
                    df_output.insert(0,input_column1, df_train[base_column1])
                    
                    
                filename=input_column+'.csv'
                #print(filename)
                #display(df_output.head(5))
                df_output.to_csv(filename,index=False)
            except KeyError:
                #print("Error:Some of these tables not present")
                print("")
                
    #zipdir='/Users/ivishnoi/Documents/DQC_2019/DQC_Pipeline1/code_base_14March/code_base_March25_latest/DQC_codebase_22May/'+path
            #print(zipdir)
    #shutil.make_archive('result_tables', 'zip', zipdir)
    #print("Results zipped")
                
    print("*Results stored-done*")
    
    print((time.clock()-start)/60)   







Before dropping empty columns, df,shape is  (1000, 82)
Column NDB.F5938DB_PROV.org_typ_cd dropped with  0.796
Column CDB.f6435dbp_cnsm_dtl.cml_prxst_typ_cd dropped with  0.991
Column CDB.f6435dbp_cnsm_dtl.cust_nm dropped with  0.992
Column CDB.f6435dbp_data_src_typ.src_cd dropped with  0.822
Column CDB.f6435dbp_cntry.cntry_cd dropped with  0.728
Column CDB.f6435dbp_ee_sts_typ.ee_sts_typ_cd dropped with  0.987
Column CDB.f6435dbp_empmt_clss_typ.empmt_clss_typ_cd dropped with  0.981
Column CDB.f6435dbp_gdr_typ.gdr_typ_cd dropped with  0.997
Column CDB.f6435dbp_lang_catgy_pref.lang_ctgy_cd dropped with  0.998
Column CDB.f6435dbp_lang_pref_typ.lang_typ_cd dropped with  0.492
Column CDB.f6435dbp_mrtl_sts_typ.mrtl_sts_typ_cd dropped with  0.99
Column CDB.f6435dbp_sbscr_rel_typ.sbscr_rel_typ_cd dropped with  0.996
Column CDB.f6435dbp_salry_typ.salry_typ_cd dropped with  0.992
Column CDB.f6435dbp_st.st_cd dropped with  0.922
Column CDB.f6435dbp_uhg_ethnc_sel_typ.uhg_ethnc_cd dropped with  0.54

*Features for test data-done*
0.016356283333333367
*Feature based scores-done*
0.01816134999999998
*Jaccard scores-done*
0.05577058333333333
*Name similarity scores-done*
0.02549093333333339
*Combining scores 1- done*
0.0006278000000000598
*Weighted scores calculation-Done*
0.002037183333333322
NDB/f5938db_h_msc.csv
Filename :  f5938db_h_msc
NDB.f5938db_h_msc
Successfully created the directory /Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/NDB_Results/NDB.f5938db_h_msc 
*Results stored-done*
0.026456616666666714
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (1000, 21)
After dropping empty columns, df,shape is   (1000, 21)
Before dropping empty columns, df,shape is  (1000, 21)
After dropping empty columns, df,shape is   (1000, 21)
### TEST DATA:  NDB/f5938db_h_accrdt_agcy_adr.csv
*Test data shape: (1000, 21)
*Reading test data-done*
0.0010071666666667293
*Features for test data-done*
0.01437093333333325
*Feature based s

*Reading test data-done*
0.0017534499999998824
*Features for test data-done*
0.01437570000000008
*Feature based scores-done*
0.019291033333333256
*Jaccard scores-done*
0.0663042666666667
*Name similarity scores-done*
0.03156891666666652
*Combining scores 1- done*
0.0007151166666666597
*Weighted scores calculation-Done*
0.0027596500000001355
NDB/f5938db_excl.csv
Filename :  f5938db_excl
NDB.f5938db_excl
Successfully created the directory /Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/NDB_Results/NDB.f5938db_excl 
*Results stored-done*
0.03175845
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (371, 22)
Column f5938db_h_optnt_srvc_catgy_head.auth_id dropped with  0.3477088948787062
Column f5938db_h_optnt_srvc_catgy_head.dlt_ind dropped with  0.3477088948787062
Column f5938db_h_optnt_srvc_catgy_head.optnt_surg_ind dropped with  0.6954177897574124
After dropping empty columns, df,shape is   (371, 19)
/Users/mm37/Documents/DQ

*Feature based scores-done*
0.02230451666666653
*Jaccard scores-done*
0.02967803333333355
*Name similarity scores-done*
0.015442633333333334
*Combining scores 1- done*
0.000425066666666396
*Weighted scores calculation-Done*
0.001324533333333496
NDB/f5938db_extr_reim_invn_sts.csv
Filename :  f5938db_extr_reim_invn_sts
NDB.f5938db_extr_reim_invn_sts
Successfully created the directory /Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/NDB_Results/NDB.f5938db_extr_reim_invn_sts 
*Results stored-done*
0.0170606166666668
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (0, 47)
After dropping empty columns, df,shape is   (0, 47)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (1000, 13)
After dropping empty columns, df,shape is   (1000, 13)
Before dropping empty columns, df,shape is  (1000, 13)
After dropping empty columns, df,shape is   (1000, 13)
### TEST DATA:  NDB/f5938db_ndc

*Results stored-done*
0.033218499999999554
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (49, 13)
After dropping empty columns, df,shape is   (49, 13)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (3, 12)
After dropping empty columns, df,shape is   (3, 12)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (1000, 22)
Column f5938db_h_prov_adr_lang.lang_wrtn_by_cd dropped with  0.954
After dropping empty columns, df,shape is   (1000, 21)
Before dropping empty columns, df,shape is  (1000, 21)
After dropping empty columns, df,shape is   (1000, 21)
### TEST DATA:  NDB/f5938db_h_prov_adr_lang.csv
*Test data shape: (1000, 21)
*Reading test data-done*
0.0010240000000000767
*Features for test data-done*
0.01435371666666659
*Feature based scores-done*
0.017788233333332924
*Jaccard scores-done*
0.03967441666666645
*Name similarity scor

...

*Results stored-done*
0.01887649999999989
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (168, 26)
After dropping empty columns, df,shape is   (168, 26)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (1000, 34)
After dropping empty columns, df,shape is   (1000, 34)
Before dropping empty columns, df,shape is  (1000, 34)
After dropping empty columns, df,shape is   (1000, 34)
### TEST DATA:  NDB/f5938db_h_ds_prov_facl_hiq_desg.csv
*Test data shape: (1000, 34)
*Reading test data-done*
0.0015397166666663983
*Features for test data-done*
0.029068366666666633
*Feature based scores-done*
0.019043683333332714
*Jaccard scores-done*
0.06381478333333349
*Name similarity scores-done*
0.03184656666666683
*Combining scores 1- done*
0.000498333333333297
*Weighted scores calculation-Done*
0.0027679000000006226
NDB/f5938db_h_ds_prov_facl_hiq_desg.csv
Filename :  f5938db_h_ds_prov_facl_hiq_desg
N

*Jaccard scores-done*
0.027863616666666265
*Name similarity scores-done*
0.01409135000000011
*Combining scores 1- done*
0.00040855000000021845
*Weighted scores calculation-Done*
0.0012270833333341594
NDB/f5938db_btch_fl_proc_stat.csv
Filename :  f5938db_btch_fl_proc_stat
NDB.f5938db_btch_fl_proc_stat
Successfully created the directory /Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/NDB_Results/NDB.f5938db_btch_fl_proc_stat 
*Results stored-done*
0.016192849999999717
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (40, 14)
After dropping empty columns, df,shape is   (40, 14)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (17, 12)
After dropping empty columns, df,shape is   (17, 12)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (3, 13)
After dropping empty columns, df,shape is   (3, 13)
/Users/mm37/Documents/DQC_Is

*Reading test data-done*
0.0016753500000002456
*Features for test data-done*
0.015993583333334036
*Feature based scores-done*
0.02439329999999984
*Jaccard scores-done*
0.07161111666666689
*Name similarity scores-done*
0.03538316666666598
*Combining scores 1- done*
0.0005131166666672016
*Weighted scores calculation-Done*
0.002991683333333602
NDB/f5938db_mass_mnt_contr_prc.csv
Filename :  f5938db_mass_mnt_contr_prc
NDB.f5938db_mass_mnt_contr_prc
Successfully created the directory /Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/NDB_Results/NDB.f5938db_mass_mnt_contr_prc 
*Results stored-done*
0.03749971666666738
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (1000, 18)
After dropping empty columns, df,shape is   (1000, 18)
Before dropping empty columns, df,shape is  (1000, 18)
After dropping empty columns, df,shape is   (1000, 18)
### TEST DATA:  NDB/f5938db_mkt_ipa_prd_dtl.csv
*Test data shape: (1000, 18)
*Reading test dat

*Features for test data-done*
0.021183266666667085
*Feature based scores-done*
0.018432283333333053
*Jaccard scores-done*
0.057710083333334224
*Name similarity scores-done*
0.028766666666666652
*Combining scores 1- done*
0.0005024499999999913
*Weighted scores calculation-Done*
0.0024067333333334774
NDB/f5938db_h_mkt.csv
Filename :  f5938db_h_mkt
NDB.f5938db_h_mkt
Successfully created the directory /Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/NDB_Results/NDB.f5938db_h_mkt 
*Results stored-done*
0.030764466666666598
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (348, 14)
After dropping empty columns, df,shape is   (348, 14)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (388, 19)
After dropping empty columns, df,shape is   (388, 19)
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (1000, 23)
After dropping empty 

*Results stored-done*
0.0170006833333332
/Users/mm37/Documents/DQC_Ishitha/DQC_codebase_6June_copy
Before dropping empty columns, df,shape is  (1000, 17)
After dropping empty columns, df,shape is   (1000, 17)
Before dropping empty columns, df,shape is  (1000, 17)
After dropping empty columns, df,shape is   (1000, 17)
### TEST DATA:  NDB/f5938db_fee_sched_mod_xcpt.csv
*Test data shape: (1000, 17)
*Reading test data-done*
0.0010114833333318529
*Features for test data-done*
0.010230416666666997
*Feature based scores-done*
0.02305811666666765
*Jaccard scores-done*
0.035141150000000225
*Name similarity scores-done*
0.01760133333333253
*Combining scores 1- done*
0.00048394999999838245
*Weighted scores calculation-Done*
0.0015002000000000255
NDB/f5938db_fee_sched_mod_xcpt.csv
Filename :  f5938db_fee_sched_mod_xcpt
NDB.f5938db_fee_sched_mod_xcpt
Successfully created the directory /Users/mm37/Documents/DQC_Ishitha//DQC_codebase_6June_copy/NDB_Results/NDB.f5938db_fee_sched_mod_xcpt 
*Results sto

In [17]:
path

'NDB/'

In [7]:
working_dir

'/Users/mm37/Documents/DQC_Ishitha/Similarity->DL/'

In [8]:
working_dir+ "OUTPUT/" + path

'/Users/mm37/Documents/DQC_Ishitha/Similarity->DL/OUTPUT/BOS.ALT_CUST'